# Molecular Interaction Mapping: From ProLIF Analysis to 3D Rendering

This notebook is divided into two parts:

- Detecting interactions in a complex protein-ligand using PROLIF
- Preparing the data for Blender

# Interaction detection with PROLIF

## Required libraries

Was tested with:

- MDAnalysis 2.6.1
- PROLIF 2.0.3

In [1]:
import MDAnalysis as mda
import prolif as plf
import json

## Prepare your Universe

In [145]:
# load topology and trajectory
u = mda.Universe(plf.datafiles.TOP, plf.datafiles.TRAJ)

# create selections for the ligand and protein
ligand_selection = u.select_atoms("resname LIG")
protein_selection = u.select_atoms("protein")
ligand_selection, protein_selection

(<AtomGroup with 79 atoms>, <AtomGroup with 4988 atoms>)

You will need the topology and trajectory files to be loaded in your Blender session once the interaction data is generated.
To get those used in this notebook you can either save your universe into dedicated files or find them in your PROLIF installation.
For the second option they were stored under the following path:

In [ ]:
/home/user/miniforge3/envs/mdanalysis/lib/python3.11/site-packages/prolif/data/
.
├── ...
├── top.pdb
├── traj.xtc
├── ...


## Fixing issues

Your data might not working working out of the box with PROLIF but thankfully their [tutorials](https://prolif.readthedocs.io/en/stable/source/tutorials.html) provide multiple solutions to make them compatible.
We won't go into these steps as we used the data used in these tutorials which are directly working.

## Detecting the interactions

Be aware that selecting a subgroup of the protein or the ligand will induce an index offset. It will work perfectly for the interaction analysis but it will complicate the preparation of the data (see the section *Convert PROLIF data into a JSON structure*).

In [4]:
# use default interactions
fp = plf.Fingerprint()
# run on a slice of the trajectory frames: from begining to end with a step of 10
fp.run(u.trajectory, ligand_selection, protein_selection)

  0%|          | 0/250 [00:00<?, ?it/s]

<prolif.fingerprint.Fingerprint: 9 interactions: ['Hydrophobic', 'HBAcceptor', 'HBDonor', 'Cationic', 'Anionic', 'CationPi', 'PiCation', 'PiStacking', 'VdWContact'] at 0x7f2a6a60c700>

# Convert PROLIF data into a JSON structure

When an interaction is detected, PROLIF returns as nested dictionnary that contains the *parent_indices*, which are the indices of the atoms involved in the interaction. For the protein, the values are useable as is but for the ligand, which is generally at the end of the file (always double check !), the indices will be reset. To fix that we need to update our indices.  

In this example we just needed to add the total number of atoms for the protein to the ligand indices. This will depend on your system. Also, if you select a subset of atoms you will need to update the indices accordingly.   

In [5]:
# Results of the first frame
fp.ifp[0]

{(ResidueId(LIG, 1, G), ResidueId(TRP, 125, A)): {'Hydrophobic': ({'indices': {'ligand': (22,), 'protein': (16,)}, 'parent_indices': {'ligand': (22,), 'protein': (1432,)}, 'distance': 3.9495952011818654},)}, (ResidueId(LIG, 1, G), ResidueId(LEU, 126, A)): {'Hydrophobic': ({'indices': {'ligand': (22,), 'protein': (13,)}, 'parent_indices': {'ligand': (22,), 'protein': (1453,)}, 'distance': 4.325043698349231},)}, (ResidueId(LIG, 1, G), ResidueId(PHE, 330, B)): {'Hydrophobic': ({'indices': {'ligand': (3,), 'protein': (12,)}, 'parent_indices': {'ligand': (3,), 'protein': (4006,)}, 'distance': 4.125894160546924},), 'VdWContact': ({'indices': {'ligand': (73,), 'protein': (16,)}, 'parent_indices': {'ligand': (73,), 'protein': (4010,)}, 'distance': 2.754668872495495},)}, (ResidueId(LIG, 1, G), ResidueId(ASP, 352, B)): {'VdWContact': ({'indices': {'ligand': (56,), 'protein': (7,)}, 'parent_indices': {'ligand': (56,), 'protein': (4368,)}, 'distance': 2.668295603212969},)}, (ResidueId(LIG, 1, G), 

In [6]:
prot_length = len(u.select_atoms("protein"))
prot_length

4988

Without going into too much details you can notice difference treatments for the storage of the different interactions:

- Hydrogen bonds which are connection between two atoms only need two indices, that of the ligand atom and of the protein atom.
- Pi-Stacking which are done between several aromatic atoms must be stored with all the aromatic atoms involved.

The structure might change in next versions but keep in mind that those indices are needed for the representation construction in Blender. Custom changes might break it if you don't know what you do.

In [ ]:
# Initialize dictionary
inter_dict = {}

# Process each frame
for frame_idx, (frame, info) in enumerate(fp.ifp.items()):
    frame_str = str(frame_idx)
    
    # Process each residue pair and their interactions
    for couple, interactions in info.items():
        # print(interactions)
        for i_type, atoms in interactions.items():
            # Map interaction types
            if i_type in ["HBAcceptor", "HBDonor"]:
                i_type = "Hydrogen_Bond"
            elif i_type in ["Cationic", "Anionic"]:
                i_type = "Salt_Bridge"
            elif "Face" in i_type:
                i_type = "PiStacking"
            
            # Initialize dictionaries if needed
            inter_dict.setdefault(i_type, {}).setdefault(frame_str, set())
            
            # Process atoms
            parent_indices = atoms[0]['parent_indices']
            lig_indices = [int(index + prot_length) for index in parent_indices['ligand']]
            prot_indices = [int(index) for index in parent_indices['protein']]
            
            if i_type == "CationPi":
                unique_lig_indices = tuple(set(lig_indices))
                unique_prot_indices = tuple(set(prot_indices))
                inter_dict[i_type][frame_str].add((unique_prot_indices, unique_lig_indices))
            # Handle special cases
            elif i_type == "PiStacking":
                unique_lig_indices = tuple(set(lig_indices))
                unique_prot_indices = tuple(set(prot_indices))
                inter_dict[i_type][frame_str].add((unique_prot_indices, unique_lig_indices))
            elif i_type == "Hydrogen_Bond":
                if len(parent_indices['ligand']) == 1 and len(parent_indices['protein']) == 2:
                    inter_dict[i_type][frame_str].add((prot_indices[1], lig_indices[0]))
                elif len(parent_indices['ligand']) == 2 and len(parent_indices['protein']) == 1:
                    inter_dict[i_type][frame_str].add((prot_indices[0], lig_indices[1]))
                else:
                    for prot_idx in prot_indices:
                        for lig_idx in lig_indices:
                            inter_dict[i_type][frame_str].add((prot_idx, lig_idx))
            else:
                for prot_idx in prot_indices:
                    for lig_idx in lig_indices:
                        inter_dict[i_type][frame_str].add((prot_idx, lig_idx))

Hydrophobic
Hydrophobic
Hydrophobic
VdWContact
VdWContact
VdWContact
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
Salt_Bridge
VdWContact
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
Hydrophobic
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Hydrophobic
VdWContact
VdWContact
VdWContact
Salt_Bridge
VdWContact
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
VdWContact
VdWContact
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
VdWContact
VdWContact
Hydrophobic
Salt_Bridge
VdWContact
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
VdWContact
VdWContact
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Hydrophobic
Hydrophobic
VdWContact
Hydrophobic
VdWContact
Salt_Bridge
VdWContact
Hydro

# Save PROLIF data in a JSON file

This JSON file is the only thing needed by the Blender extension to build your interactions. If you use the same interaction types and conserve a coherent structure, you can generate your file without using PROLIF. 

In [144]:
# Convert sets to lists for JSON serialization
inter_dict_serializable = {k: {frame: list(data) for frame, data in v.items()} for k, v in inter_dict.items()}

# Save inter_dict to a compact JSON file
with open('interactions_test.json', 'w') as json_file:
    json.dump(inter_dict_serializable, json_file, separators=(',', ':'))

In [139]:
# Read the JSON file
with open('interactions_test.json', 'r') as json_file:
    inter_dict_serializable = json.load(json_file)

In [112]:
test_dict = {}

for i_type, frames in inter_dict_serializable.items():
    test_dict[i_type] = {}
    for frame, pairs in frames.items():
        test_dict[i_type][frame] = pairs

In [ ]:
blender -b protein_ligand.blend -s 0 -e 250 --python ../../../../Blender/Python\ Scripting/Render/MN_sessions.py -a